In [2]:
!pip install wordninja

You should consider upgrading via the 'C:\Users\Ahlaam\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
# Web scraper for Reddit data

# data analysis imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# NLP imports
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import wordninja

import requests
import time
import random

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ahlaam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ahlaam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# create user agent
headers = {"User-agent" : "wingednova"} # set user agent to reddit account username
url_1 = "https://www.reddit.com/r/depression.json"

res = requests.get(url_1, headers=headers)
res.status_code

# scraper function
def reddit_scrape(url_string, number_of_scrapes, output_list):
    #scraped posts outputted as lists
    after = None 
    for _ in range(number_of_scrapes):
        if _ == 0:
            print("SCRAPING {}\n--------------------------------------------------".format(url_string))
            print("<<<SCRAPING COMMENCED>>>") 
            print("Downloading Batch {} of {}...".format(1, number_of_scrapes))
        elif (_+1) % 5 ==0:
            print("Downloading Batch {} of {}...".format((_ + 1), number_of_scrapes))
        
        if after == None:
            params = {}
        else:
            #THIS WILL TELL THE SCRAPER TO GET THE NEXT SET AFTER REDDIT'S after CODE
            params = {"after": after}             
        res = requests.get(url_string, params=params, headers=headers)
        if res.status_code == 200:
            the_json = res.json()
            output_list.extend(the_json["data"]["children"])
            after = the_json["data"]["after"]
        else:
            print(res.status_code)
            break
        time.sleep(random.randint(1,6))
    
    print("<<<SCRAPING COMPLETED>>>")
    print("Number of posts downloaded: {}".format(len(output_list)))
    print("Number of unique posts: {}".format(len(set([p["data"]["name"] for p in output_list]))))

# remove any repeat posts
def create_unique_list(original_scrape_list, new_list_name):
    data_name_list=[]
    for i in range(len(original_scrape_list)):
        if original_scrape_list[i]["data"]["name"] not in data_name_list:
            new_list_name.append(original_scrape_list[i]["data"])
            data_name_list.append(original_scrape_list[i]["data"]["name"])
    #CHECKING IF THE NEW LIST IS OF SAME LENGTH AS UNIQUE POSTS
    print("LIST NOW CONTAINS {} UNIQUE SCRAPED POSTS".format(len(new_list_name)))

In [4]:
# scraping r/SuicideWatch data
suicide_data = []
reddit_scrape("https://www.reddit.com/r/SuicideWatch.json", 50, suicide_data)

suicide_data_unique = []
create_unique_list(suicide_data, suicide_data_unique)

# add r/SuicideWatch data to dataframe
suicide_watch = pd.DataFrame(suicide_data_unique)
suicide_watch["state"] = 1
suicide_watch.head()

# scraping r/depression data
depression_data = []
reddit_scrape("https://www.reddit.com/r/depression.json", 50, depression_data)

depression_data_unique = []
create_unique_list(depression_data, depression_data_unique)

# add r/depression data to dataframe
depression = pd.DataFrame(depression_data_unique)
depression["state"] = 0
depression.head()

# saving data
suicide_watch.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\suicide_watch.csv', index = False)
depression.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\depression.csv', index = False)

# creating combined CSV
depression = pd.read_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\depression.csv')
suicide_watch = pd.read_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\suicide_watch.csv')

dep_columns = depression[["title", "selftext", "author",  "num_comments", "state","url"]]
sui_columns = suicide_watch[["title", "selftext", "author",  "num_comments", "state","url"]]

combined_data = pd.concat([dep_columns, sui_columns],axis=0, ignore_index=True)  
combined_data["selftext"].fillna("emptypost",inplace=True)
combined_data.head()
combined_data.isnull().sum()

# saving combined CSV
combined_data.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\suicide_vs_depression.csv', index = False)

SCRAPING https://www.reddit.com/r/SuicideWatch.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1251
Number of unique posts: 974
LIST NOW CONTAINS 974 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/depression.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1230
Number of unique posts: 980
LIST NOW CONTAINS 980 UNIQUE SCRAPED POSTS


### Scraping r/bulimia, r/AnorexiaNervosa -- test subreddits #1

In [5]:
# scraping r/bulimia data
bulimia_data = []
reddit_scrape("https://www.reddit.com/r/bulimia.json", 50, bulimia_data)

bulimia_data_unique = []
create_unique_list(bulimia_data, bulimia_data_unique)

# add r/bulimia data to dataframe
bulimia = pd.DataFrame(bulimia_data_unique)
bulimia.head()

# scraping r/AnorexiaNervosa data
anorexia_data = []
reddit_scrape("https://www.reddit.com/r/AnorexiaNervosa.json", 50, anorexia_data)

anorexia_data_unique = []
create_unique_list(anorexia_data, anorexia_data_unique)

# add r/AnorexiaNervosa data to dataframe
anorexia = pd.DataFrame(anorexia_data_unique)
anorexia.head()

# saving data
bulimia.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\bulimia.csv', index = False)
anorexia.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\anorexia.csv', index = False)

SCRAPING https://www.reddit.com/r/bulimia.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1242
Number of unique posts: 992
LIST NOW CONTAINS 992 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/AnorexiaNervosa.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1239
Number of unique posts: 912
LIST NOW CONTAINS 912 UNIQUE SCRAPED POSTS


### Scraping additional subreddits: r/eating_disorders (training), r/EDAnonymousAdults

In [4]:
# scraping r/eating_disorders data
ED_train_data = []
reddit_scrape("https://www.reddit.com/r/eating_disorders.json", 50, ED_train_data)

ED_train_data_unique = []
create_unique_list(ED_train_data, ED_train_data_unique)

# add r/eating_disorders data to dataframe
ED_train = pd.DataFrame(ED_train_data_unique)
ED_train.head()

# scraping r/EDAnonymousAdults data
EDanon_adults_data = []
reddit_scrape("https://www.reddit.com/r/EdAnonymousAdults.json", 50, EDanon_adults_data)

EDanon_adults_data_unique = []
create_unique_list(EDanon_adults_data, EDanon_adults_data_unique)

# add r/EDAnonymousAdults data to dataframe
EDanon_adults = pd.DataFrame(EDanon_adults_data_unique)
EDanon_adults.head()

# saving data
ED_train.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\ED_train.csv', index = False)
EDanon_adults.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\EDanon_adults.csv', index = False)

SCRAPING https://www.reddit.com/r/eating_disorders.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1242
Number of unique posts: 942
LIST NOW CONTAINS 942 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/EdAnonymousAdults.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1247
Number of unique posts: 997
LIST NOW CONTAINS 997 UNIQUE SCRAPED POSTS


### Scraping recovery subreddits: r/EatingDisorders, r/AnorexiaRecovery, r/BulimiaRecovery, r/fuckeatingdisorders, r/EDRecovery_public

In [5]:
########## scraping r/EatingDisorders data
EDisorders_data = []
reddit_scrape("https://www.reddit.com/r/EatingDisorders.json", 50, EDisorders_data)

EDisorders_data_unique = []
create_unique_list(EDisorders_data, EDisorders_data_unique)

# add r/EatingDisorders data to dataframe
EDisorders = pd.DataFrame(EDisorders_data_unique)
EDisorders.head()

########## scraping r/AnorexiaRecovery data
ANRecovery_data = []
reddit_scrape("https://www.reddit.com/r/AnorexiaRecovery.json", 50, ANRecovery_data)

ANRecovery_data_unique = []
create_unique_list(ANRecovery_data, ANRecovery_data_unique)

# add r/AnorexiaRecovery data to dataframe
ANRecovery = pd.DataFrame(ANRecovery_data_unique)
ANRecovery.head()

########## scraping r/BulimiaRecovery
BNRecovery_data = []
reddit_scrape("https://www.reddit.com/r/BulimiaRecovery.json", 50, BNRecovery_data)

BNRecovery_data_unique = []
create_unique_list(BNRecovery_data, BNRecovery_data_unique)

# add r/BulimiaRecovery data to dataframe
BNRecovery = pd.DataFrame(BNRecovery_data_unique)
BNRecovery.head()

########## scraping r/fuckeatingdisorders
fED_data = []
reddit_scrape("https://www.reddit.com/r/fuckeatingdisorders.json", 50, fED_data)

fED_data_unique = []
create_unique_list(fED_data, fED_data_unique)

# add r/fuckeatingdisorders data to dataframe
fED = pd.DataFrame(fED_data_unique)
fED.head()

########## scraping r/EDRecovery_public
EDRecovery_data = []
reddit_scrape("https://www.reddit.com/r/EDRecovery_public.json", 50, EDRecovery_data)

EDRecovery_data_unique = []
create_unique_list(EDRecovery_data, EDRecovery_data_unique)

# add r/EDRecovery_public data to dataframe
EDRecovery = pd.DataFrame(EDRecovery_data_unique)
EDRecovery.head()

# saving data
EDisorders.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\recovery\\EDisorders.csv', index = False)
ANRecovery.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\recovery\\ANRecovery.csv', index = False)
BNRecovery.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\recovery\\BNRecovery.csv', index = False)
fED.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\recovery\\fED.csv', index = False)
EDRecovery.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\recovery\\EDRecovery.csv', index = False)

SCRAPING https://www.reddit.com/r/EatingDisorders.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1245
Number of unique posts: 893
LIST NOW CONTAINS 893 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/AnorexiaRecovery.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1235
Number of unique posts: 984
LIST NOW CONTAINS 984 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/BulimiaRecovery.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1179
Number of unique posts: 141
LIST NOW CONTAINS 141 UNIQUE SCRAPED POSTS
SCRAPING https://www.reddit.com/r/fuckeatingdisorders.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1250
Number of unique posts: 99

In [6]:
########## scraping r/CasualConversation data
casconv_data = []
reddit_scrape("https://www.reddit.com/r/CasualConversation.json", 50, casconv_data)

casconv_data_unique = []
create_unique_list(casconv_data, casconv_data_unique)

# add r/EatingDisorders data to dataframe
casconv = pd.DataFrame(casconv_data_unique)
casconv["state"] = 2
casconv.head()

# saving data
casconv.to_csv('C:\\Users\\Ahlaam\\Desktop\\EP Sem 7\\BTP\\new-data\\control\\casconv.csv', index = False)

SCRAPING https://www.reddit.com/r/CasualConversation.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1235
Number of unique posts: 809
LIST NOW CONTAINS 809 UNIQUE SCRAPED POSTS
